In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re
import json 
from pandas.api.types import is_numeric_dtype


In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [ ]:
resultadoAmsa = sf.query_all(
    """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

In [393]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false  
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(14500, 8)

In [394]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7873, 5)

In [395]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(14506, 10)

In [396]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [397]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1918613, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [398]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1512511, 4)

In [399]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1512511, 13)

In [400]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [401]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [402]:
# pacientes_empresas22.shape

In [403]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [404]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [405]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [406]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [407]:
Pacientes_Archivo.head(10)

,poliza,RUTASEGURADO,DV,rut2,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,FNAC,nacimiento,sexo,...,TELEFONO,RELACION,DESDE,HASTA,NOMBRE,apellidos,nombres,email,celular,VIGENCIA
0,340007641,17919107,5,179191075,23785026,2,237850262,2011-10-28 00:00:00,2011-10-28,M,...,962444115,HI,2022-01-01,NaN,Berney Fuente - Alba Lorenzo,Berney Fuente,- Alba Lorenzo,berney322@gmail.com,NaN,2023-12-31 00:00:00
1,340019966,25792665,6,257926656,25792665,6,257926656,1967-04-08 00:00:00,1967-04-08,M,...,NaN,AS,2022-09-01,NaN,Maignan Jean - Claude,Maignan Jean,- Claude,msuyon@ats.cl,NaN,2024-09-30 00:00:00
2,340013161,19308143,6,193081436,21581328,2,215813282,1996-10-16 00:00:00,1996-10-16,F,...,NaN,CO,2024-01-01,NaN,Llanos Ruíz - Díaz Mariana,Llanos Ruíz,- Díaz Mariana,brunodelamarelorca@gmail.com,NaN,2024-12-31 00:00:00
3,340022591,14452649,K,14452649K,14452649,K,14452649K,1974-08-03 00:00:00,1974-08-03,M,...,958292887,AS,2022-09-01,NaN,Villalon Mc- Intosc Francisco Javier,Villalon Mc,- Intosc Francisco Javier,francisco.villalon@alstomgroup.com,958292887,2023-12-31 00:00:00
4,340002351,13433652,8,134336528,25192362,0,251923620,2015-11-14 00:00:00,2015-11-14,F,...,64009 7141289,HI,2015-11-14,NaN,Montenegro Saint- Jean Luciana,Montenegro Saint,- Jean Luciana,wmontene@cisco.com,968315285,2023-12-31 00:00:00
5,340021371,18159063,7,181590637,15563239,9,155632399,1983-02-15 00:00:00,1983-02-15,M,...,956999450,CO,2022-12-01,NaN,Hasbun Von- Kunowsky Nicolas,Hasbun Von,- Kunowsky Nicolas,laura.alonso@tecnofarma.cl,956999450,2024-07-31 00:00:00
6,340007856,17313813,K,17313813K,24841019,1,248410191,2014-12-31 00:00:00,2014-12-31,M,...,+56 41989845,HI,2021-12-28,NaN,Pizarro Tiozzo - Lyon Agustin Antonio,Pizarro Tiozzo,- Lyon Agustin Antonio,mtiozzolyo@bancochile.cl,941989845,2024-11-30 00:00:00
7,340007856,17313813,K,17313813K,25493270,1,254932701,2016-08-31 00:00:00,2016-08-31,F,...,+56 41989845,HI,2021-12-28,NaN,Pizarro Tiozzo - Lyon Amalia Constanza,Pizarro Tiozzo,- Lyon Amalia Constanza,mtiozzolyo@bancochile.cl,941989845,2024-06-30 00:00:00
8,340007856,17313813,K,17313813K,26520535,6,265205356,2018-10-12 00:00:00,2018-10-12,M,...,+56 41989845,HI,2021-12-28,NaN,Pizarro Tiozzo - Lyon Andres Alonso,Pizarro Tiozzo,- Lyon Andres Alonso,mtiozzolyo@bancochile.cl,941989845,NaN
9,340020033,18391545,2,183915452,18391545,2,183915452,1992-12-20 00:00:00,1992-12-20,F,...,994484886,AS,2021-11-01,NaN,Rodriguez Sainte - Marie Maria Isabel,Rodriguez Sainte,- Marie Maria Isabel,isabelrsm@gmail.com,994484886,NaN


In [408]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [409]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,RUTASEGURADO,DV,rut2,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,FNAC,nacimiento,sexo,...,RELACION,DESDE,HASTA,NOMBRE,apellidos,nombres,email,celular,VIGENCIA,rut
0,340007641,17919107,5,179191075,23785026,2,237850262,2011-10-28 00:00:00,2011-10-28,M,...,HI,2022-01-01,NaN,Berney Fuente - Alba Lorenzo,Berney Fuente,- Alba Lorenzo,berney322@gmail.com,NaN,2023-12-31 00:00:00,237850262
1,340019966,25792665,6,257926656,25792665,6,257926656,1967-04-08 00:00:00,1967-04-08,M,...,AS,2022-09-01,NaN,Maignan Jean - Claude,Maignan Jean,- Claude,msuyon@ats.cl,NaN,2024-09-30 00:00:00,257926656
2,340013161,19308143,6,193081436,21581328,2,215813282,1996-10-16 00:00:00,1996-10-16,F,...,CO,2024-01-01,NaN,Llanos Ruíz - Díaz Mariana,Llanos Ruíz,- Díaz Mariana,brunodelamarelorca@gmail.com,NaN,2024-12-31 00:00:00,215813282
3,340022591,14452649,K,14452649K,14452649,K,14452649K,1974-08-03 00:00:00,1974-08-03,M,...,AS,2022-09-01,NaN,Villalon Mc- Intosc Francisco Javier,Villalon Mc,- Intosc Francisco Javier,francisco.villalon@alstomgroup.com,958292887,2023-12-31 00:00:00,14452649K
4,340002351,13433652,8,134336528,25192362,0,251923620,2015-11-14 00:00:00,2015-11-14,F,...,HI,2015-11-14,NaN,Montenegro Saint- Jean Luciana,Montenegro Saint,- Jean Luciana,wmontene@cisco.com,968315285,2023-12-31 00:00:00,251923620


In [410]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,RUTASEGURADO,DV,rut2,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,FNAC,nacimiento,sexo,...,RELACION,DESDE,HASTA,NOMBRE,apellidos,nombres,email,celular,VIGENCIA,rut
0,340007641,17919107,5,179191075,23785026,2,237850262,2011-10-28 00:00:00,2011-10-28,M,...,HI,2022-01-01,NaN,Berney Fuente - Alba Lorenzo,Berney Fuente,- Alba Lorenzo,berney322@gmail.com,NaN,2023-12-31 00:00:00,237850262
1,340019966,25792665,6,257926656,25792665,6,257926656,1967-04-08 00:00:00,1967-04-08,M,...,AS,2022-09-01,NaN,Maignan Jean - Claude,Maignan Jean,- Claude,msuyon@ats.cl,NaN,2024-09-30 00:00:00,257926656
2,340013161,19308143,6,193081436,21581328,2,215813282,1996-10-16 00:00:00,1996-10-16,F,...,CO,2024-01-01,NaN,Llanos Ruíz - Díaz Mariana,Llanos Ruíz,- Díaz Mariana,brunodelamarelorca@gmail.com,NaN,2024-12-31 00:00:00,215813282
3,340022591,14452649,K,14452649K,14452649,K,14452649K,1974-08-03 00:00:00,1974-08-03,M,...,AS,2022-09-01,NaN,Villalon Mc- Intosc Francisco Javier,Villalon Mc,- Intosc Francisco Javier,francisco.villalon@alstomgroup.com,958292887,2023-12-31 00:00:00,14452649K
4,340002351,13433652,8,134336528,25192362,0,251923620,2015-11-14 00:00:00,2015-11-14,F,...,HI,2015-11-14,NaN,Montenegro Saint- Jean Luciana,Montenegro Saint,- Jean Luciana,wmontene@cisco.com,968315285,2023-12-31 00:00:00,251923620


In [411]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()

Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\1368669443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


In [412]:
def siPrimerCero(y):
    if y == "00":
        return "Nones"
    if y[0] == "0":
        y = y[1:]
    return y

In [413]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))

In [414]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))

Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\620742920.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\620742920.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [415]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(761319, 35)

In [416]:
Pacientes_con_id.head()

,poliza,RUTASEGURADO,DV,rut2,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,FNAC,nacimiento,sexo,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340007641,17919107,5,179191075,23785026,2,237850262,2011-10-28 00:00:00,2011-10-28,M,...,True,0018c00002mSKdAAAW,237850262,Berney Fuente Alba Lorenzo,RUT,None,None,None,Hombre,None
1,340019966,25792665,6,257926656,25792665,6,257926656,1967-04-08 00:00:00,1967-04-08,M,...,True,001RN000001SwapYAC,257926656,Jean Claude Maignan,RUT,None,None,None,Hombre,None
2,340013161,19308143,6,193081436,21581328,2,215813282,1996-10-16 00:00:00,1996-10-16,F,...,True,001RN0000057fMvYAI,215813282,Díaz Mariana Llanos Ruíz,RUT,None,None,None,Mujer,None
3,340022591,14452649,K,14452649K,14452649,K,14452649K,1974-08-03 00:00:00,1974-08-03,M,...,True,0018c00002cX0AqAAK,14452649K,Francisco Javie Villalon Mc-Intosh,RUT,None,None,None,Hombre,None
4,340002351,13433652,8,134336528,25192362,0,251923620,2015-11-14 00:00:00,2015-11-14,F,...,True,0018c00002bIcEZAA0,251923620,Luciana Montenegro Saint- Jean,RUT,None,None,None,Mujer,None


In [417]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [418]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
""" if ("rutT" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Titulares"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    ) """
    
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\2256497620.py:93: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [419]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [420]:
""" print(dia_hoy) """



' print(dia_hoy) '

In [421]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [422]:
campaña_seleccionada.shape

(980466, 13)

In [423]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

' if campaña != "MetLife":\n    pacientes_en_campaña = pd.merge(\n        left=campaña_seleccionada,\n        right=Pacientes_con_id,\n        how="left",\n        left_on="Id Account",\n        right_on="Id Account Paciente",\n    )\n    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"\n    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"\n    pacientes_en_campaña.to_excel(\n        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False\n    )\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False\n    )\n '

In [424]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

' if campaña != "MetLife1":\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    if ("poliza" in campaña_en_pacientes.columns) == True:\n        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)\n\n    campaña_en_pacientes = campaña_en_pacientes.rename(\n        columns={"Programas_por_Empresas__c": "programas por empresa"}\n    )\n    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]\n    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[\n        "programas por empresa"\n    ]\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False\n    ) '

In [425]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [426]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [446]:
nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
#campaña = "BCI Seguros Colectivos"
#Pais = "Chile"
# Pais = "México"

rutes = pd.read_excel(nuevosrutes)

In [447]:
#derty = rutes.to_numpy().transpose().tolist()

In [448]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
rutesi.shape

(761319, 9)

In [449]:
rutesi.head()

,Rut,Id Account Paciente,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,237850262,0018c00002mSKdAAAW,Berney Fuente Alba Lorenzo,RUT,None,None,None,Hombre,None
1,257926656,001RN000001SwapYAC,Jean Claude Maignan,RUT,None,None,None,Hombre,None
2,215813282,001RN0000057fMvYAI,Díaz Mariana Llanos Ruíz,RUT,None,None,None,Mujer,None
3,14452649K,0018c00002cX0AqAAK,Francisco Javie Villalon Mc-Intosh,RUT,None,None,None,Hombre,None
4,251923620,0018c00002bIcEZAA0,Luciana Montenegro Saint- Jean,RUT,None,None,None,Mujer,None


In [450]:

rutesi = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [451]:
rutesi.head()

,Rut,Id Account Paciente,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Pacientes_Programas__c,...,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,237850262,0018c00002mSKdAAAW,Berney Fuente Alba Lorenzo,RUT,None,None,None,Hombre,None,a1Y8c00000Cy2owEAB,...,MetLife,0018c00002mSLnUAAW,Servicios Integrales Cacciuttolo Ltda.,Plan Agema - ADM,MetLife,Servicios Integrales Cacciuttolo Ltda.,340007641,None,EMP10777,A-10323
1,257926656,001RN000001SwapYAC,Jean Claude Maignan,RUT,None,None,None,Hombre,None,a1YRN0000007QA12AM,...,MetLife,001RN000001W2uIYAS,ATS Maquinarias Ltda.,EXSTDEB,MetLife,ATS Maquinarias Ltda.,340019966,None,EMP11243,A-10797
2,215813282,001RN0000057fMvYAI,Díaz Mariana Llanos Ruíz,RUT,None,None,None,Mujer,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14452649K,0018c00002cX0AqAAK,Francisco Javie Villalon Mc-Intosh,RUT,None,None,None,Hombre,None,a1Y8c00000Dy5bmEAB,...,MetLife,0018c00002cXcjLAAS,Alstom Chile S.A.,Sin Holding Asociado,MetLife,Alstom Chile S.A.,340022591,None,EMP6444,A-2188
4,251923620,0018c00002bIcEZAA0,Luciana Montenegro Saint- Jean,RUT,None,None,None,Mujer,None,a1Y8c00000CmKaQEAV,...,MetLife,0018c00002axQZ5AAM,Cisco Systems Chile S.A.,Sin Holding Asociado,MetLife,Cisco Systems Chile S.A.,340002351,None,EMP5949,A-0076


In [452]:
rutesi.to_excel(
    "F:/Empresas SF/reportes/rutesi1dd1.xlsx", index=False
)

KeyboardInterrupt: 

In [434]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf
simpler_sf.simple_salesforce()


In [435]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [436]:
query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query)


100%|██████████| 1/1 [03:15<00:00, 195.01s/it]


In [437]:
nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
#campaña = "BCI Seguros Colectivos"
#Pais = "Chile"
# Pais = "México"

rutes = pd.read_excel(nuevosrutes)

In [438]:
df1.head()

,Id,Account.Name,RutContact__c
0,07k8c00000RGWPAAA5,Astara Chile SPA,182204846
1,07k8c00000RGWPBAA5,Astara Chile SPA,18396206K
2,07k8c00000RGWPCAA5,Astara Chile SPA,184262452
3,07k8c00000RGWPDAA5,Astara Chile SPA,184429772
4,07k8c00000RGWPKAA5,Serv. Bien. Trabajadores Finning Chile S.A.,233986291


In [439]:
df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace("-", "")# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\4047109371.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1["rut"] = df1["rut"].str.replace(".", "")


,Id,Account.Name,RutContact__c,rut
0,07k8c00000RGWPAAA5,Astara Chile SPA,182204846,182204846
1,07k8c00000RGWPBAA5,Astara Chile SPA,18396206K,18396206K
2,07k8c00000RGWPCAA5,Astara Chile SPA,184262452,184262452
3,07k8c00000RGWPDAA5,Astara Chile SPA,184429772,184429772
4,07k8c00000RGWPKAA5,Serv. Bien. Trabajadores Finning Chile S.A.,233986291,233986291


In [440]:
rutes["rut"] = rutes["Rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()
rutes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15112\992701059.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rutes["rut"] = rutes["rut"].str.replace(".", "")


,Rut,rut
0,99997060,99997060
1,99995343,99995343
2,99987812,99987812
3,99978252,99978252
4,99949139,99949139


In [441]:
rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
)

In [442]:
rutesi.to_excel(
    "F:/Empresas SF/reportes/rutesi1dd1.xlsx", index=False
)